이 장에서는 여러분이 부동산 회사에 막 고용된 데이터 과학자라고 가정하고 예제 프로젝트의 처음부터 끝까지 진행해보려한다. 진행할 주요 단계는 다음과 같다.
1. 큰 그림을 본다.
2. 데이터를 구한다.
3. 데이터로부터 통찰을 얻기 위해 탐색하고 시각화한다.
4. 머신러닝 알고리즘을 위해 데이터를 준비한다.
5. 모델을 선택하고 훈련시킨다.
6. 모델을 상세하게 조정한다.
7. 솔루션을 제시한다.
8. 시스템을 론칭하고 모니터링하고 유지 보수한다.

# 실제 데이터로 작업하기

머신러닝을 배울 때는 인공적으로 만들어진 데이터셋이 아닌 실제 데이터로 실험해보는 것이 가장 좋다. 다행히 여러 분야에 걸쳐 공개된 데이터셋이 아주 많다. 다음은 데이터를 구하기 좋은 곳이다.

<ul>
    <li>유명한 공개 데이터 저장소
        <ul>
            <li>UC 얼바인<sup>lrvine</sup> 머신러닝 저장소(<a href='http://archive.ics.uci.edu/ml' target='blank'>http://archive.ics.uci.edu/ml</a>)</li>
            <li>캐글<sup>Kaggle</sup> 데이터셋(<a href='http://www.kaggle.com/datasets' target='blank'>http://www.kaggle.com/datasets</a>)</li>
            <li>아마존 AWS 데이터셋(<a href='https://registry.opendata.aws' target='blank'>https://registry.opendata.aws</a>)</li>
        </ul></li>
    <li>메타 포털(공개 데이터 저장소가 나열되어 있다)
        <ul>
            <li>데이터 포털<sup>Data Portals</sup>(<a href='http://dataportals.org' target='blank'>http://dataportals.org</a>)</li>
            <li>오픈 데이터 모니터<sup>Open Data Monitor</sup>(<a href='http://opendatamonitor.eu' target='blank'>http://opendatamonitor.eu</a>)</li>
            <li>퀀들<sup>Quandl</sup>(<a href='http://quandl.com' target='blank'>http://quandl.com</a>)</li>
        </ul></li>
    <li>인기 있는 공개 데이터 저장소가 나열되어 있는 다른 페이지
        <ul>
            <li>위키백과 머신러닝 데이터셋 목록(<a href='https://goo.gl.SJHN2k' target='blank'>https://goo.gl.SJHN2k</a>)</li>
            <li>Quora.com(<a href='https://homl.info/10' target='blank'>https://homl.info/10</a>)</li>
            <li>데이터셋 서브레딧<sup>subreddit</sup>(<a href='http://www.reddit.com/r/datasets' target='blank'>http://www.reddit.com/r/datasets</a>)</li>
        </ul></li>
    </ul>

이 장에서는 StatLib 저장소에 있는 캘리포니아 주택 가격<sup>California Housing Prices</sup> 데이터셋을 사용한다(그림 2-1). 이 데이터셋은 1990년 캘리포니아 인구조사 데이터를 기반으로 한다. 최근 데이터는 아니지만(그 당시의 가격이라면 여러분도 베이 에어리어<sup>Bay Area</sup>에 꽤 좋은 집을 살 수 있다) 학습용으로 아주 좋기 때문에 최근 데이터라고 생각하겠다. 교육 목적으로 사용하기 위해 범주형 특성을 추가하고 몇 개 특성을 제외했다.

<strong>그림 2-1</strong> 캘리포니아 주택 가격

# 큰 그림 보기

맨 처음 할 일은 캘리포니아 인구조사 데이터를 사용해 캘리포니아의 주택 가격 모델을 만드는 것이다. 이 데이터는 캘리포니아의 블록 그룹<sup>block group[1](#p01)</sup>(구역)마다 인구<sup>population</sup>, 중간 소득<sup>median income</sup>, 중간 주택 가격<sup>median housing price</sup> 등을 담고 있다.

이 데이터로 모델을 학습시켜서 다른 측정 데이터가 주어졌을 때 구역의 중간 주택 가격을 예측해야 한다.

## 문제 정의

상사에게 첫 번째로 할 질문은 '비지니스의 목적이 정확히 무엇인가요?'이다. 아마도 모델 만들기가 최종 목적은 아닐 것이다. 회사에서는 이 모델을 어떻게 사용해 이익을 얻으려고 할까? 목적을 아는 것은 문제를 어떻게 구성할지, 어떤 알고리즘을 선택할지, 모델 평가에 어떤 성능 지표를 사용할지, 모델 튜닝을 위해 얼마나 노력을 투여할지 결정하기 때문에 아주 중요하다.

상사가 이 모델의 출력(구역의 중간 주택 가격에 대한 예측)이 여러 가지 다른 <strong>신호</strong><sup>signal[2](#p02)</sup>와 함께 다른 머신러닝 시스템에 입력으로 사용된다고 한다. 뒤따르는 시스템이 해당 지역에 투자할 가치가 있는지 결정하는데 이 결정이 수익에 직결되기 때문에 올바르게 예측하는 것은 매우 중요하다.

<div style="border:1px solid; padding:10px;">
    <div style="text-align: center"><strong>파이프라인</strong></div><br>
    데이터 처리 <strong>컴포넌트</strong><sup>component</sup>들이 연속되어 있는 것을 데이터 <strong>파이프라인</strong><sup>pipeline</sup>이라고 한다. 머신러닝 시스템은 데이터를 조작하고 변환할 일이 많아 파이프라인을 사용하는 일이 매우 흔하다.
    
보통 컴포넌트들은 비동기적으로 동작한다. 각 컴포넌트는 많은 데이터를 추출해 처리하고 그 결과를 다른 데이터 저장소로 보낸다. 그러면 일정 시간 후 파이프라인의 다음 컴포넌트가 그 데이터를 추출해 자신의 출력 결과를 만든다. 각 컴포넌트는 완전히 독립적이다. 즉, 컴포넌트 사이의 인터페이스는 데이터 저장소뿐이다. 이는 (데이터 흐름도 덕분에) 시스템을 이해하기 쉽게 만들고, 각 팀은 각자의 컴포넌트에 집중할 수 있다. 한 컴포넌트가 다운되더라도 하위 컴포넌트는 문제가 생긴 컴포넌트의 마지막 출력을 사용해 (적어도 한동안은) 평상시와 같이 계속 동작할 수 있다. 그래서 시스템이 매우 견고해진다.
  
한편 모니터링이 적절히 되지 않으면 고장 난 컴포넌트를 한동안 모를 수 있다. 데이터가 만들어진지 오래 되면 전체 시스템의 성능이 떨어진다.
</div>

# 각주

<a name='p01'>1</a> 블록 그룹은 미국 인구조사국에서 샘플 데이터를 발표하는 데 사용하는 최소한의 지리적 단위로 하나의 블록 그룹은 보통 600~3,000명의 인구를 나타낸다).

<a name='p02'>2</a> 머신러닝 시스템에 주입하는 정보를 클로드 섀넌(Claude Shannon)의 정보 이론을 따라 '신호'라고 부른다. 그는 전기 통신 기술을 개선하기 위해 벨 연구소에서 이 이론을 개발했다. 그의 이론에 따르면 신호/잡음 비율이 높은 것이 좋다.